In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
from pathlib import Path 
from PIL import Image
from scipy import ndimage, misc
import numpy as np
import cv2
import os
import math

In [31]:
def remove_noise(imgray):
    ori_imgray = imgray.copy()
    row, col = imgray.shape
    resolution = 4
    
    # invertcolorはいろんな実装方法があるが、例えばnumpyのbroadcastを使えば以下の1行でかけます
    # imgray = 255 - imgray
    # ただし、このあとの処理を見るに、cv2.threshold(imgray, 245, 255, cv2.THRESH_BINARY_INV) でも同じ結果になりそうです
    # (removenoisefun.pngの画像が反転すること以外)
    ret, thresh = cv2.threshold(imgray, 245, 255, cv2.THRESH_BINARY_INV)
    # 今後使わない変数は作成さずに、 _ で受けるという手段があります
    _, contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)    

    while (len(contours)>1):
        print("resolution",resolution)
        resolution += 1
        imgray = ori_imgray.copy()        

        imgray = resolution_image(imgray, resolution)
        
        ret, thresh = cv2.threshold(imgray,245,255,cv2.THRESH_BINARY_INV)
        _, contours, _ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    
    # 最初から len(contours) <= 1の場合whileに入らないので、ここでcntを受け取る必要がある
    # また、contoursが空の場合は[0]でアクセスできないので注意
    #get the coordinate from land
    cnt = contours[0] if len(contours)>0 else np.array([[]])
    #print ('contours',contours)       

    # (N, 1, 2) のndarrayを N要素の(x, y)に変換するコードは例えばこのような書き方もあります 
    land_cor = cnt.reshape(-1, 2).tolist()
    
    test_output = os.path.join('./tochi_test37/'+'removenoisefun'+'.png' )
    print("output_path is", test_output)
    # 今までの結果と互換性を保つため、imgrayを反転した画像を保存します
    cv2.imwrite(test_output, 255-imgray)

    return imgray, land_cor, contours 

In [27]:
from itertools import product

def resolution_image(imgray, resolution):    
    row, col = imgray.shape
    
    # ピクセル値が255のピクセルに対して、その周辺を255にする処理はdilationという処理でかけます
    mask = (imgray == 255).astype(np.uint8) * 255
    kernel = np.ones((3, 3),np.uint8)
    dilated = cv2.dilate(mask, kernel, iterations=resolution)
    imgray[dilated.astype(np.bool)] = 255
    
    # 任意のピクセル値(0~254)を膨張するのは少しややこしいが、numpyのwhereとindexing, broadcastingを使った方が多少簡潔になる
    for _ in range (resolution):
        ys, xs = np.where(imgray != 255)
        v = imgray[ys, xs]
        for dy, dx in product([-1, 0, 1], repeat=2):
            imgray[np.clip(ys+dy, 0, row-1), np.clip(xs+dx, 0, col-1)] = v

    # in-place処理ではあるが、一応変換後のnumpy.ndarrayも返却した方がpythonっぽい
    return imgray

In [19]:
from itertools import product

In [22]:
list(product([-1, 0, 1], repeat=2))

[(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 0), (0, 1), (1, -1), (1, 0), (1, 1)]

In [5]:
def road_detection(new_imgray,ori_imgray):
    # numpyのbroadcastingを使えば簡潔に書けます
    # new_imgray!=255でTrueかFalseが入った配列ができて、
    # fancy_indexingとbroadcastという機能で一括で変更できます
    # 下のセルに例を書いてみました
    ori_imgray[new_imgray!=255] = 255
 
    return ori_imgray

In [6]:
x = np.random.rand(3, 3)
print(x)
print(x>0.5)
x[x>0.5] = 1
print(x)

[[0.17100255 0.24311748 0.38688406]
 [0.07308525 0.51422236 0.47207915]
 [0.26423818 0.28563264 0.41202207]]
[[False False False]
 [False  True False]
 [False False False]]
[[0.17100255 0.24311748 0.38688406]
 [0.07308525 1.         0.47207915]
 [0.26423818 0.28563264 0.41202207]]


In [7]:
def hough_linedetection(ori_imgray):
 
    backtorgb = cv2.cvtColor(ori_imgray,cv2.COLOR_GRAY2RGB)
    edges = cv2.Canny(backtorgb,100,200,apertureSize = 3)
    minLineLength = 50
    maxLineGap = 10
    threshold = 15
 
    lines = cv2.HoughLinesP(edges,1,np.pi/180,threshold,minLineLength,maxLineGap)
  
    return lines


In [8]:
def median_filter(land_cor):
    median_result = ndimage.median_filter(imgray, size=(3, 3))
    
    return median_result

In [9]:
def getPoints(x1, y1, x2, y2, gp_lst):

    if (x1==x2):
        for y in range (y1,y2+1):
                gp_lst.append((x1,y))
    else:
    
        m = (y2-y1) / (x2-x1)
        b = y1 - m * x1
        for x in range(x1, x2+1):
            y = m * x + b
            y=int(y)
            gp_lst.append((x,y))
    #print (gp_lst)
    return gp_lst

In [10]:
def line_dist(x1,y1,x2,y2):
    x=abs(x1-x2)
    y=abs(y1-y2)
    sum_line=math.sqrt((x**2)+(y**2))
    return sum_line

In [11]:
"""
この関数については、リファクタリングの余地はあるのですが、それ以前に
returnを書くインデントがおかしいと思いました。
下に print('必ず一回しかこのprintは通りません') を書いたとおり、これは最初のループで必ずreturnしてしまいます
"""
def getMinLengthLineSegment(point_lst):

    min_point=[]
    min_length = line_dist(point_lst[0][0], point_lst[0][1], point_lst[1][0], point_lst[1][1])
    min_point.append(point_lst[0])
    min_point.append(point_lst[1])
    print("min_point is",min_point)

    for i in range(len(point_lst)):
        if (i != len(point_lst)-1):
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[i+1]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line<min_length):
                min_length=sum_line
                min_point[0]=point_lst[i]
                min_point[1]=point_lst[i+1]
                print("min_point is",min_point)
        else:
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[0]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line<min_length):
                min_length=sum_line
                min_point[0]=point_lst[i]
                min_point[1]=point_lst[0]
                print("min_point is",min_point)
                
        print ("min_legth is",min_length) 
        print ("min_point is",min_point) 
        print('必ず一回しかこのprintは通りません')
        return min_length, min_point

In [12]:
def getMaxLengthLineSegment(point_lst):

    max_point=[]
    max_length = line_dist(point_lst[0][0], point_lst[0][1], point_lst[1][0], point_lst[1][1])
    max_point.append(point_lst[0])
    max_point.append(point_lst[1])

    for i in range(len(point_lst)):
        if (i != len(point_lst)-1):
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[i+1]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line>max_length):
                max_length=sum_line
                max_point[0]=point_lst[i]
                max_point[1]=point_lst[i+1]

        else:
            x1, y1 = point_lst[i]
            x2, y2 = point_lst[0]
            sum_line = line_dist(x1,y1,x2,y2)

            if (sum_line>max_length):
                max_length=sum_line
                max_point[0]=point_lst[i]
                max_point[1]=point_lst[0]
                
                
    print ("max_legth is",max_length) 
    print ("max_point is",max_point)             
    return max_length, max_point

In [13]:
def Gravity_Center(point_lst):
    # numpyを使うと簡潔に書けます
    GC_x, GC_y = np.mean(np.asarray(point_lst), axis=0).astype(np.int32)
    print(GC_x)
    print(GC_y)
    
    return GC_x,GC_y
        #print('point_lst is',point_lst[i])

In [14]:
def dist(x1,y1, x2,y2, x3,y3): # x3,y3 is the point
    px = x2-x1
    py = y2-y1

    something = px*px + py*py

    u =  ((x3 - x1) * px + (y3 - y1) * py) / float(something)

    if u > 1:
        u = 1
    elif u < 0:
        u = 0

    x = x1 + u * px
    y = y1 + u * py

    dx = x - x3
    dy = y - y3

    # Note: If the actual distance does not matter,
    # if you only want to compare what this function
    # returns to other results of this function, you
    # can just return the squared distance instead
    # (i.e. remove the sqrt) to gain a little performance

    dist = math.sqrt(dx*dx + dy*dy)
    print (dist)

    return dist

In [15]:
def enhance_image(imgray):
    print('ENHACE')
    #median_result=median_filter(imgray)
    #output_path = os.path.join('./tochi_test37/'+'median_filter'+'.bmp' )
    #cv2.imwrite(output_path, median_result)
    #print('median print finish')

In [39]:
def run(img_path):
    """
    img_path: path_lib.Path object. path for input image
    """
    
    # note: opencvのimreadはpath_lib.Pathを受け付けられない
    im = cv2.imread(str(img_path))
    # 元々のコードのようにファイルの存在チェックをするのも悪くはないが、
    # 「ファイルは存在するが、画像ファイルではない」といったケースも考えると、
    # cv2.imreadしてしまって、結果をチェックしてしまったほうが良い
    if im is None:
        raise ValueError('cannot read image: {}'.format(img_path))

    #Gray tone
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    # todo row, colは消す
    row, col = imgray.shape

    # 1. ori_imgrayなどの変数はこのスコープで持っていても使わないので、関数にわたすときに作れば十分
    # 2. row, colはremove_noiseの中でori_imgrayから取得できる
    # 3. 受け取る変数名は意味がわかりやすいものにした方がよい
    noise_removed_imgray, land_cor, contours = remove_noise(imgray.copy())
    road_imgray=road_detection(noise_removed_imgray, imgray.copy())
    
    if len(land_cor) == 0:
        print('土地が見つかりませんでした {}'.format(img_path))
        return
     
    min_length, min_point=getMinLengthLineSegment(land_cor)
    max_length, max_point=getMaxLengthLineSegment(land_cor)
 
    #hough
    backtorgb = cv2.cvtColor(road_imgray,cv2.COLOR_GRAY2RGB)
    hough_lines=hough_linedetection(road_imgray)
    
    for x in range(0, len(hough_lines)):
        for x1,y1,x2,y2 in hough_lines[x]:
            cv2.line(backtorgb,(x1,y1),(x2,y2),(0,255,0),2)
    
    hough_output = os.path.join('./tochi_test37/'+img_path.stem+'_hough'+'.png' )
 
    cv2.imwrite(hough_output, backtorgb)
    ret,thresh = cv2.threshold(noise_removed_imgray,127,255,0)
    
    #get all point from hough
    gp_lst=[]
    for i in range (0,len(hough_lines)):
         for x1,y1,x2,y2 in hough_lines[i]:
                getPoints(x1, y1, x2, y2, gp_lst)
                
    
    #gravity of road 
    GC_x, GC_y=Gravity_Center(gp_lst)
    print ("GC_x, GC_y",GC_x, GC_y)
    
    cv2.line(backtorgb,(0,0),(GC_x, GC_y),(0,255,0),2)
    
    gravity_output = os.path.join('./tochi_test37/'+'gravity_output'+'.png' )
    cv2.imwrite(gravity_output, backtorgb)
    
    #point to line distance
    
    x1,y1=max_point[0]
    x2,y2=max_point[1]
    
    #print ('max_point',type(max_point))
    #print ('min_point',type(min_point))
    
    dist(x1,y1, x2,y2, GC_x, GC_y)
    
    x1,y1=min_point[0]
    x2,y2=min_point[1]
    dist(x1,y1, x2,y2, GC_x, GC_y)        
    
  

In [40]:
run(Path('tochi/C62145A23.png'))

resolution 4
output_path is ./tochi_test37/removenoisefun.png
min_point is [[533, 188], [532, 189]]
min_legth is 1.4142135623730951
min_point is [[533, 188], [532, 189]]
必ず一回しかこのprintは通りません
max_legth is 15.0
max_point is [[311, 259], [311, 274]]
108
346
GC_x, GC_y 108 346
215.3903433304288
452.133829745132


In [41]:
files = Path('tochi').glob('*.png')
for f in files:
    run(f)

resolution 4
resolution 5
resolution 6
resolution 7
resolution 8
resolution 9
resolution 10
resolution 11
resolution 12
resolution 13
resolution 14
resolution 15
resolution 16
resolution 17
resolution 18
resolution 19
resolution 20
resolution 21
resolution 22
resolution 23
resolution 24
resolution 25
resolution 26
resolution 27
resolution 28
resolution 29
resolution 30
resolution 31
resolution 32
resolution 33
resolution 34
resolution 35
resolution 36
resolution 37
resolution 38
resolution 39
resolution 40
resolution 41
resolution 42
resolution 43
output_path is ./tochi_test37/removenoisefun.png
min_point is [[239, 238], [238, 239]]
min_legth is 1.4142135623730951
min_point is [[239, 238], [238, 239]]
必ず一回しかこのprintは通りません
max_legth is 135.0
max_point is [[374, 238], [239, 238]]
363
344
GC_x, GC_y 363 344
106.0
163.1318485152424
resolution 4
output_path is ./tochi_test37/removenoisefun.png
min_point is [[58, 346], [57, 347]]
min_legth is 1.4142135623730951
min_point is [[58, 346], [57, 3

resolution 5
resolution 6
resolution 7
resolution 8
resolution 9
resolution 10
resolution 11
resolution 12
resolution 13
resolution 14
resolution 15
resolution 16
resolution 17
resolution 18
resolution 19
resolution 20
resolution 21
resolution 22
resolution 23
resolution 24
resolution 25
output_path is ./tochi_test37/removenoisefun.png
min_point is [[188, 149], [188, 233]]
min_legth is 84.0
min_point is [[188, 149], [188, 233]]
必ず一回しかこのprintは通りません
max_legth is 219.0
max_point is [[224, 549], [443, 549]]
321
91
GC_x, GC_y 321 91
458.0
145.09651959988565
resolution 4
resolution 5
resolution 6
resolution 7
resolution 8
resolution 9
resolution 10
resolution 11
resolution 12
output_path is ./tochi_test37/removenoisefun.png
min_point is [[322, 33], [321, 34]]
min_legth is 1.4142135623730951
min_point is [[322, 33], [321, 34]]
必ず一回しかこのprintは通りません
max_legth is 411.0
max_point is [[376, 450], [376, 39]]
321
485
GC_x, GC_y 321 485
65.19202405202648
451.0
resolution 4
resolution 5
resolution 6
re

resolution 61
resolution 62
resolution 63
resolution 64
resolution 65
resolution 66
resolution 67
resolution 68
resolution 69
resolution 70
resolution 71
resolution 72
resolution 73
resolution 74
resolution 75
resolution 76
resolution 77
resolution 78
resolution 79
resolution 80
resolution 81
resolution 82
resolution 83
resolution 84
resolution 85
resolution 86
resolution 87
resolution 88
resolution 89
resolution 90
resolution 91
resolution 92
resolution 93
resolution 94
resolution 95
resolution 96
resolution 97
resolution 98
resolution 99
resolution 100
resolution 101
resolution 102
resolution 103
resolution 104
resolution 105
resolution 106
resolution 107
resolution 108
resolution 109
resolution 110
resolution 111
resolution 112
resolution 113
resolution 114
resolution 115
resolution 116
resolution 117
resolution 118
resolution 119
resolution 120
resolution 121
resolution 122
resolution 123
resolution 124
resolution 125
resolution 126
resolution 127
resolution 128
resolution 129
reso

output_path is ./tochi_test37/removenoisefun.png
min_point is [[38, 265], [38, 267]]
min_legth is 2.0
min_point is [[38, 265], [38, 267]]
必ず一回しかこのprintは通りません
max_legth is 226.0
max_point is [[31, 272], [31, 498]]
329
169
GC_x, GC_y 329 169
315.29827148273426
306.42617381679395
resolution 4
resolution 5
resolution 6
resolution 7
resolution 8
resolution 9
resolution 10
resolution 11
resolution 12
resolution 13
resolution 14
resolution 15
resolution 16
resolution 17
resolution 18
resolution 19
resolution 20
resolution 21
resolution 22
resolution 23
resolution 24
resolution 25
resolution 26
resolution 27
resolution 28
resolution 29
resolution 30
resolution 31
resolution 32
resolution 33
resolution 34
resolution 35
resolution 36
resolution 37
resolution 38
resolution 39
resolution 40
resolution 41
resolution 42
resolution 43
resolution 44
resolution 45
resolution 46
resolution 47
resolution 48
resolution 49
resolution 50
resolution 51
resolution 52
resolution 53
resolution 54
resolution 55


resolution 25
resolution 26
resolution 27
resolution 28
resolution 29
resolution 30
resolution 31
resolution 32
resolution 33
resolution 34
resolution 35
resolution 36
resolution 37
resolution 38
resolution 39
resolution 40
resolution 41
resolution 42
resolution 43
resolution 44
resolution 45
resolution 46
resolution 47
resolution 48
resolution 49
resolution 50
resolution 51
resolution 52
resolution 53
resolution 54
output_path is ./tochi_test37/removenoisefun.png
min_point is [[201, 41], [201, 44]]
min_legth is 3.0
min_point is [[201, 41], [201, 44]]
必ず一回しかこのprintは通りません
max_legth is 373.0
max_point is [[195, 47], [195, 420]]
276
388
GC_x, GC_y 276 388
81.0
352.08095659947304
resolution 4
output_path is ./tochi_test37/removenoisefun.png
min_point is [[479, 150], [478, 151]]
min_legth is 1.4142135623730951
min_point is [[479, 150], [478, 151]]
必ず一回しかこのprintは通りません
max_legth is 263.0
max_point is [[537, 419], [537, 156]]
183
119
GC_x, GC_y 183 119
355.92836357896516
296.73051747334654
res

resolution 5
resolution 6
resolution 7
resolution 8
resolution 9
resolution 10
resolution 11
resolution 12
resolution 13
resolution 14
resolution 15
resolution 16
resolution 17
resolution 18
resolution 19
resolution 20
output_path is ./tochi_test37/removenoisefun.png
min_point is [[113, 246], [113, 278]]
min_legth is 32.0
min_point is [[113, 246], [113, 278]]
必ず一回しかこのprintは通りません
max_legth is 432.0
max_point is [[54, 536], [486, 536]]
284
69
GC_x, GC_y 284 69
467.0
246.1097316239242
resolution 4
output_path is ./tochi_test37/removenoisefun.png
min_point is [[240, 236], [240, 239]]
min_legth is 3.0
min_point is [[240, 236], [240, 239]]
必ず一回しかこのprintは通りません
max_legth is 279.0
max_point is [[352, 522], [352, 243]]
313
178
GC_x, GC_y 313 178
75.8023746329889
93.23625904121208
resolution 4
resolution 5
resolution 6
output_path is ./tochi_test37/removenoisefun.png
min_point is [[54, 198], [54, 201]]
min_legth is 3.0
min_point is [[54, 198], [54, 201]]
必ず一回しかこのprintは通りません
max_legth is 383.0
max

resolution 65
resolution 66
resolution 67
resolution 68
resolution 69
resolution 70
resolution 71
resolution 72
resolution 73
resolution 74
resolution 75
resolution 76
resolution 77
resolution 78
resolution 79
output_path is ./tochi_test37/removenoisefun.png
min_point is [[239, 252], [239, 253]]
min_legth is 1.0
min_point is [[239, 252], [239, 253]]
必ず一回しかこのprintは通りません
max_legth is 217.0
max_point is [[205, 429], [422, 429]]
293
403
GC_x, GC_y 293 403
26.0
159.42396306703708


In [ ]:
output_dir='./tochi'
fp=open('./tochi/image_list.txt','r')
line =  fp.readline()


while line:
   
    line = line.strip()
    
    myfile = './tochi/' + line
    
    print (myfile)
    run(myfile,line)
    line =  fp.readline()


print("out of while")    

    
    

In [ ]:
print("all program is done")